# Growing Degree Days
This program is for calculating growing degree days in a green house tomato. 

In [1]:
import pandas as pd
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
import sys
import datetime as dt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bkcharts import * 
from bokeh.palettes import PuBu

from bokeh.models.widgets import Tabs, Panel
import pandas, datetime
from bokeh.models import HoverTool,Range1d,ColumnDataSource,HoverTool
from bokeh.layouts import gridplot
from time import gmtime,strftime
from calendar import timegm
from bokeh.layouts import row, column
from bokeh.embed import components
from time import gmtime,strftime


In [2]:
# Connection to MySQL Database to get the data
#conn = pymysql.connect(host = 'localhost', port= 8889, user ='root', passwd = 'root', db = 'test_DB')
conn = pymysql.connect(host = '', port= 3306, user ='', passwd = '', db = '')

cursor1 = conn.cursor()
cursor2 = conn.cursor()

cursor1.execute('SELECT sdate,ROUND((max_temp+min_temp)/2,1), sensor FROM daily_temp ORDER BY sdate ASC')
#cursor1.execute('SELECT sdate,ROUND((max_temp+min_temp)/2-10,1), sensor FROM daily_temp ORDER BY sdate ASC')
cursor2.execute('SELECT DISTINCT block,vid,planting, cutting FROM toguchi_plant WHERE  planting IN (SELECT Planting FROM toguchi_plant)')
result1 = cursor1.fetchall()
result2 = cursor2.fetchall()

df1 = pd.DataFrame( [[ij for ij in i] for i in result1] )
df2 = pd.DataFrame( [[ij for ij in i] for i in result2] )

df1.rename(columns={0: 'Date', 1:'Mean_T', 2:'sensor'}, inplace=True);
df2.rename(columns={0: 'block', 1: 'vid', 2: 'startHarv', 3: 'endHarv'}, inplace=True);
if cursor1 and conn:
    cursor1.close()
if cursor2 and conn:
    cursor2.close()
    conn.close()

In [3]:
df1.head()

,Date,Mean_T,sensor
0,2017-06-01,27.5,02:00:00:28:8c:e6
1,2017-06-02,23.7,02:00:00:28:8c:e6
2,2017-06-03,23.4,02:00:00:28:8c:e6
3,2017-06-04,24.0,02:00:00:28:8c:e6
4,2017-06-05,24.1,02:00:00:28:8c:e6


In [4]:
df2.head()

,block,vid,startHarv,endHarv
0,A,v1,2017-06-11,2017-08-02
1,F,v1,2016-04-20,2016-08-20
2,E,v1,2016-05-22,2016-09-12
3,D,v1,2016-06-21,2016-10-12
4,C,v1,2016-07-22,2016-12-04


In [5]:
df2_A = df2.query('block == "A" & vid == "v1"')
df1_A = df1.query('sensor == "02:00:00:28:8c:e6"')

df2_B = df2.query('block == "B" & vid == "v1"')
df1_B = df1.query('sensor == "70:ee:50:27:2e:da"')

df2_C = df2.query('block == "C" & vid == "v1"')
df1_C= df1.query('sensor == "03:00:00:05:05:30"')

df2_D = df2.query('block == "D" & vid == "v1"')
df1_D = df1.query('sensor == "03:00:00:02:c5:f8"')

df2_E = df2.query('block == "E" & vid == "v1"')
df1_E = df1.query('sensor == "02:00:00:27:39:82"')

df2_F = df2.query('block == "F" & vid == "v1"')
df1_F = df1.query('sensor == "03:00:00:04:f2:00"')

In [6]:
df2_A.head()

,block,vid,startHarv,endHarv
0,A,v1,2017-06-11,2017-08-02


In [7]:
#Drop
df = df1_A.drop('sensor',1)
clean_df1_B = df1_B.drop('sensor',1)
clean_df1_C = df1_C.drop('sensor',1)
clean_df1_D = df1_D.drop('sensor',1)
clean_df1_E = df1_E.drop('sensor',1)
clean_df1_F = df1_F.drop('sensor',1)

In [9]:
clean_df2_A = df2_A.drop(['block','vid'], 1)

In [10]:
clean_df2_A

,startHarv,endHarv
0,2017-06-11,2017-08-02


In [11]:
startHarv_A= clean_df2_A['startHarv'].iloc[0]
endHarv = dt.date.today()

In [12]:
full_range = pd.date_range(startHarv_A, endHarv)
df.set_index(['Date'], inplace=True)

In [13]:
df_intepl = df.sort_index().reindex(full_range).interpolate(method='linear')

In [14]:
def HDH (Tcurr,Tref=10.0):
    if Tref <= Tcurr:
        return ((Tcurr-Tref))
    else:
        return (0)

df_intepl['GDD'] = df_intepl.apply(lambda row: HDH(row['Mean_T']),axis=1)

In [15]:
df_intepl = df_intepl.reset_index(drop=False)

In [16]:
df_intepl['Cum_GDD'] = df_intepl.GDD.cumsum()


In [18]:
#outputs in Jupyter    
output_notebook()
#Create the source data with formats for date
Date1 = df.index
GDD = df.GDD
Cum_GDD = df.Cum_GDD
source = ColumnDataSource(
    data=dict(
        GDD = GDD,
        Cum_GDD = Cum_GDD,
        Date1 = pd.to_datetime(df['index'], format='%Y-%m-%d'),
    )
)
#Hover with specfied tooltips
hover4 = HoverTool(
    tooltips="""
         <div style="background: #FFFFFF;">
            <span style="font-size: 18px; color: blue;">CGDD, @Cum_GDD{int}</span><br />
        </div>
    """)

# Create figures
p1 = figure(x_axis_label='日付', x_axis_type='datetime',
            y_axis_label='積算温度',
            background_fill_color='#BFE6F6',
            tools=[hover4, 'xpan','reset','previewsave'],
            plot_width=1000,
            logo=None)
p1.line(x='Date1', y='Cum_GDD',source=source,line_width=1,line_color="blue")
p1.circle('Date1','Cum_GDD',source=source)
p1.xaxis.axis_label_text_font_size = "20pt"
p1.yaxis.axis_label_text_font_size = "20pt"

#Create
tab1 = Panel(child=p1, title='積算温度')

# Using Returned TAB create layout
layout = Tabs(tabs=[tab1])
output_file('toguchi_CGDD.html')
show(layout)

Loading BokehJS ...